In [1]:
# https://machinelearningmastery.com/multi-step-time-series-forecasting-long-short-term-memory-networks-python/
import pandas as pd
from datetime import datetime

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import math
from matplotlib import pyplot

import numpy as np

np.random.seed(7)

In [2]:
# transform series into train and test sets for supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    return agg


In [3]:
# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq):
	# extract raw values
	raw_values = series.values
	
    # transform data to be stationary
	diff_values = raw_values
	diff_values = diff_values.reshape(len(diff_values), 1)
	
    # rescale values to -1, 1
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaled_values = scaler.fit_transform(diff_values)
	scaled_values = scaled_values.reshape(len(scaled_values), 1)
	
    # transform into supervised learning problem X, y
	supervised = series_to_supervised(scaled_values, n_lag, n_seq)
	supervised_values = supervised.values
	
    # split into train and test sets
	train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
	return scaler, train, test

In [4]:
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]

    X = X.reshape(X.shape[0], 1, X.shape[1])

    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(X, y, epochs=nb_epoch, batch_size=n_batch, verbose=2, shuffle=False)

    return model

In [5]:
# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))
    
    # make forecast
    forecast = model.predict(X, batch_size=n_batch)
    
    # convert to array
    return [x for x in forecast[0, :]]
 
# evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
    forecasts = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

In [6]:
# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler):
    inverted = list()
    for i in range(len(forecasts)):
        
        # create array from forecast
        forecast = np.array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
        
        # invert scaling
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]

        inverted.append(inv_scale)

    return inverted

In [7]:
# load dataset
logreturns = 'data/final.csv'
series = pd.read_csv(logreturns, usecols=['Exchange.Date', 'logreturns'], header=0, index_col=0, squeeze=True)

# configure
n_lag = 1
n_seq = 3 #  number of periods forecast
n_test = 10
n_epochs = 5
n_batch = 1
n_neurons = 1

# prepare data
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)

# fit model
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

# make forecasts
forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)

# inverse transform forecasts and test
forecasts = inverse_transform(series, forecasts, scaler)

Epoch 1/5
4551/4551 - 5s - loss: 0.0120
Epoch 2/5
4551/4551 - 4s - loss: 0.0084
Epoch 3/5
4551/4551 - 4s - loss: 0.0083
Epoch 4/5
4551/4551 - 4s - loss: 0.0083
Epoch 5/5
4551/4551 - 4s - loss: 0.0083


In [12]:
test_df = pd.read_csv('data/final.csv', usecols=['logreturns', 'Close'])
last_train = test_df['Close'].values[-n_test-1]
test_compare = test_df['Close'].values[-n_test:]

price_forecast = np.exp(np.cumsum(forecasts[0]) + math.log(last_train)) # forecasts[0] is the first forecast from the last training value

print(price_forecast)
print(test_compare)

[1202.8978 1205.0426 1205.2523]
[1224.56 1229.45 1235.91 1235.13 1242.04 1249.1  1245.63 1243.38 1254.09
 1253.63]


In [13]:
test_df.tail(11)

,Close,logreturns
4553,1202.26,-0.003902
4554,1224.56,0.018378
4555,1229.45,0.003985
4556,1235.91,0.005241
4557,1235.13,-0.000631
4558,1242.04,0.005579
4559,1249.10,0.005668
4560,1245.63,-0.002782
4561,1243.38,-0.001808
4562,1254.09,0.008577
